In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_AM

No dicts found; please check load_dicts...


In [2]:
xi_list = GLS_Apr_weekday_AM.xi_list
P = GLS_Apr_weekday_AM.P
L = GLS_Apr_weekday_AM.L  # dimension of xi

314

In [3]:
size(P, 1), size(P, 2)

(56,314)

In [4]:
using JuMP
using Ipopt

mGLSJulia = Model(solver=IpoptSolver())

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum(p[i,j] for j = 1:size(P,2)) == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum(p[i,l] * lam[i] for i = 1:size(P,1)) == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum(p[1,j] for j = 1:size(P,2)))  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    70022
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    52752

Total number of variables............................:    17640
                     variables with only lower bounds:    17640
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [5]:
getvalue(lam)

56-element Array{Float64,1}:
 286.69 
 254.87 
 188.763
 369.817
 432.451
 471.472
 251.074
 383.43 
 321.019
 164.803
 391.854
 589.998
 549.857
   ⋮    
 601.957
 257.922
 381.071
 722.36 
 246.031
 231.047
 243.589
 392.221
 138.057
 253.285
 384.251
 202.434

In [6]:
getObjectiveValue(mGLSJulia)

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in getObjectiveValue(::JuMP.Model, ::Vararg{JuMP.Model,N}) at ./deprecated.jl:30
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading In[6], in expression starting on line 1


1.0000000000000278

In [7]:
GLS_Apr_weekday_AM.saveDemandVec(getvalue(lam))